<a href="https://colab.research.google.com/github/ArlexMR/CEE_694_Python_Lecture/blob/main/Notebooks/03_HydroDataRetriev%26Manip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Retrieval and Hydrological Applications

This notebook shows a basic aplication of the retrieval and manipulation of daily discharge data using the `dataretrieval`  Python Package, and the libraries `Pandas` and `pymannkendall`.

Unlike previous notebooks, this one is mainly demonstrative. You won't need to make major modifications to the code. However, feel free to play around with it and try your own analyses.  

At the end, you will find some exercises to replicate this analysis with new data.


# Install the Packages

The following code installs the packages required. If running in a Jupyter notebook, this must be done only the first time.

In [ ]:
!pip install dataretrieval
!pip install pymannkendall

# Import Libraries

In [ ]:
from dataretrieval import nwis # For getting the data
from IPython.display import display # Just a fancy 'plot()' for dataFrames

import pandas as pd # For tabular data manipulation
import numpy as np
import matplotlib.pyplot as plt # For creating charts
import pymannkendall as mk


# What is dataretrieval?
Dataretrieval is a Python alternative to USGS-R's dataRetrieval package for obtaining USGS or EPA water quality data, streamflow data, and metadata directly from web services. Note that dataretrieval is an alternative to the R package, not a port, in that it reproduces the functionality of the R package but its organization and functionality often differ. The Python version also expands upon its predecessor by including capability to pull data from a variety of web portals besides NWIS and STORET.  

**Input Arguments:**

* **sites** (string or list of strings): A list of USGS site identifiers for which to retrieve data.
* **parameterCd** (list of strings): A list of USGS parameter codes for which to retrieve data. See: https://help.waterdata.usgs.gov/codes-and-parameters/parameters
* **statCd** (list of strings): A list of USGS statistic codes for which to retrieve data. See: https://help.waterdata.usgs.gov/code/stat_cd_nm_query?stat_nm_cd=%25&fmt=html
* **start** (string): The beginning date for a period for which to retrieve data. If the waterdata parameter startDT is supplied, it will overwrite the start parameter.
* **end** (string): The ending date for a period for which to retrieve data. If the waterdata parameter endDT is supplied, it will overwrite the end parameter.  


source: https://github.com/USGS-python/dataretrieval




**Some other retrieval functions are:**

Function | Service
---------|--------
`get_iv` | Instantaneous (15 min) Values
`get_dv` | Daily Values
`get_stats` | Statistics
`get_discharge_peaks` | Peak Flow
`get_discharge_measurements` | Discharge Measurements
`get_gwlevels`| Groundwater levels
`get_qwdata` | Water quality

# Get Data

In [ ]:
# Set the parameters needed to retrieve data
siteNumber    = "03298000"
parameterCode = "00060" # Discharge cfs
startDate     = "1900-01-01"
endDate       = "2021-09-30"

# Retrieve the data
(dailyStreamflow, info) = nwis.get_dv(sites    = siteNumber,
                                   parameterCd = parameterCode,
                                   start       = startDate,
                                   end         = endDate
                                   )


`nwis.get_dv` returns a tuple whose first element is the data itself and the second element is the metadata. Both are Pandas `DataFrames`   

# Explore the data

### Tabular data and metadata

In [ ]:
print('Data')
display(dailyStreamflow.head())

print('\nMetadata: \n')
display(info.site_info()[0])

### Look for missing values

The  methods `info()` and `describe()` provides basic information of the Data Frame

In [ ]:
print('BASIC INFO:')
dailyStreamflow.info()

print('\nBASIC STATISTICS:')
dailyStreamflow.describe()


### Plot

Here we use `matplotlib.pyplot` which was imported as `plt`.

Here you can take a look at the variety of charts that can be created with `matplotlib`:  

https://matplotlib.org/stable/gallery/index.html

Another popular visualization library for Python is `seaborn` which is based on `matplotlib` and provides a high-level interface for beautiful charts:  

https://seaborn.pydata.org/examples/index.html


Here We will start with two basic `matplotlib` functions:

`plt.figure()` creates the canvas for a new figure. Several parameters can be included. In this case, just the `figsize` is defined.  
See: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.figure.html

`plt.plot(x,y, ... )` plots $y$ vs. $x$ lines or markers. If only one array is provided it's used as $y$ and the $x$ is inferred
see: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html  


In [ ]:
plt.figure(figsize = (10,3))
plt.plot(dailyStreamflow['00060_Mean'])

### Data flags
Let's check the number and location of each flag in the time series

In [ ]:
# Print unique values
print(dailyStreamflow['00060_Mean_cd'].unique())
# Print percentage of values for each flag
dailyStreamflow.groupby('00060_Mean_cd').count()/len(dailyStreamflow)*100


In [ ]:
# Plot data with flags

# split data according to flag
flag_A = dailyStreamflow.loc[dailyStreamflow['00060_Mean_cd'] == 'A']
flag_AR = dailyStreamflow.loc[dailyStreamflow['00060_Mean_cd'] == 'A, R']
flag_Ae = dailyStreamflow.loc[dailyStreamflow['00060_Mean_cd'] == 'A, e']

# plot each series with different color
plt.figure(figsize = (15,3))
plt.plot(flag_A['00060_Mean'],'.g', label = 'Flag: A')
plt.plot(flag_AR['00060_Mean'],'.b', label = 'Flag: A, R')
plt.plot(flag_Ae['00060_Mean'],'.r', label = 'Flag: A, e')
plt.legend(loc = 'upper center', ncol = 3 )



A list of (better) colors can be found here: https://matplotlib.org/stable/gallery/color/named_colors.html

# Calculate Seasonality

Seasonality refers to the typical variability along the year. Thus the mean discharge value for each of the twelve months must be estimated. Here, also the 5% and 95% quantiles are included. this is done with two simple steps:  
1. Add a column with the month
2. Use `groupby()` to estimate statistics for each of the unique values in the month column

In [ ]:
dailyStreamflow.head()
# Sometimes is convenient to have the date column as a normal column, besides having it in the index
dailyStreamflow['Date'] = dailyStreamflow.index
# In this case this date column is used to get the month for each data point
dailyStreamflow['month'] = dailyStreamflow['Date'].dt.month

# now let's estimate the mean, quantiles 5% and 95%, and number of samples for each month
seasonalStreamFlow = pd.DataFrame()
seasonalStreamFlow['mean_dschg'] = dailyStreamflow.groupby('month')['00060_Mean'].mean()
seasonalStreamFlow['q5'] = dailyStreamflow.groupby('month')['00060_Mean'].quantile(0.05)
seasonalStreamFlow['q95'] = dailyStreamflow.groupby('month')['00060_Mean'].quantile(0.95)
seasonalStreamFlow['count'] = dailyStreamflow.groupby('month')['00060_Mean'].count()
seasonalStreamFlow

In [ ]:
# ... and plot the results
plt.figure(figsize = (8,3))

# This creates the line with markers
plt.plot(seasonalStreamFlow['mean_dschg'],
         '-o',
         c='k',
         markerfacecolor = 'tab:red',
         markersize = 8,
         label = 'Mean daily discharge'
         )

# This creates a colored area
plt.fill_between(seasonalStreamFlow.index,
                 y1 = seasonalStreamFlow['q95'],
                 y2 = seasonalStreamFlow['q5'],
                 color = 'tab:grey',
                 alpha = 0.3,
                 label = '90% band'
                 )
plt.legend()

plt.xticks(ticks=range(1,13),
           labels = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],
           fontsize = 10
           );
plt.xlim(1,12)
plt.yticks(fontsize = 10)
plt.ylabel('Discharge (cfs)',fontsize = 13)


# Is there a trend?

Let's look for trends in annual data. In this case we take 4 characteristics discharges for each year (min, max, mean, Quantile 95%) and plot them to look for temporal trend.

The method `resample()` is useful for this task

In [ ]:
dailyStreamflow.head()
annual_Flow = pd.DataFrame()
annual_Flow['min'] = dailyStreamflow['00060_Mean'].resample('1Y').min()
annual_Flow['max'] = dailyStreamflow['00060_Mean'].resample('1Y').max()
annual_Flow['mean'] = dailyStreamflow['00060_Mean'].resample('1Y').mean()
annual_Flow['q95'] = dailyStreamflow['00060_Mean'].resample('1Y').quantile(0.95)
annual_Flow['count'] = dailyStreamflow['00060_Mean'].resample('1Y').count()
annual_Flow = annual_Flow[annual_Flow['count']/365>0.9] # remove years with less than 90% of the data
annual_Flow



In [ ]:
# a simple plot just to see what we got

plt.figure(figsize = (10,6))

plt.subplot(2,2,1)
plt.plot(annual_Flow['mean'],'.')
plt.title('Mean')

plt.subplot(2,2,2)
plt.plot(annual_Flow['min'],'.')
plt.title('Min')

plt.subplot(2,2,3)
plt.plot(annual_Flow['max'],'.')
plt.title('Max')

plt.subplot(2,2,4)
plt.plot(annual_Flow['q95'],'.')
plt.title('q95')

plt.tight_layout()


## Let's include a *Moving Average* or *Rolling Mean*

In [ ]:

rolling_annual_stats = annual_Flow.rolling(9, center = True ).agg({'min':'mean',
                                            'max':'mean',
                                            'mean':'mean',
                                            'q95':'mean',
                                            'count':'min'
                                            })
rolling_annual_stats.head(9)

In [ ]:
# ...and make a better plot
plt.figure(figsize = (13,7), facecolor = 'w')

plt.subplot(4,1,1)
plt.plot(annual_Flow['max'],'or')
plt.plot(rolling_annual_stats['max'],'k')
plt.ylabel('Annual\nMax. (cfs)', fontsize = 12)
plt.xticks(color ='w')
plt.yticks(fontsize = 10)


plt.subplot(4,1,2)
plt.plot(annual_Flow['mean'],'or')
plt.plot(rolling_annual_stats['mean'],'k')
plt.ylabel('Annual\nMean (cfs)', fontsize = 12)
plt.xticks(color ='w')
plt.yticks(fontsize = 10)

plt.subplot(4,1,3)
plt.plot(annual_Flow['min'],'or')
plt.plot(rolling_annual_stats['min'],'k')
plt.ylabel('Annual\nMin (cfs)', fontsize = 12)
plt.xticks(color ='w')
plt.yticks(fontsize = 10)


plt.subplot(4,1,4)
plt.plot(annual_Flow['q95'],'or')
plt.plot(rolling_annual_stats['q95'],'k')
plt.ylabel('Quantile\n95% (cfs)', fontsize = 12)
plt.yticks(fontsize = 10)
plt.xticks(fontsize = 10)

plt.suptitle('Discharge Trends Floyds Fork', fontsize = 15 )
# plt.tight_layout(rect = (0,0,.95,0.95) )
# plt.tight_layout()


## Let's apply a Mann-Kendall test to the annual data

In [ ]:
mk_test = mk.original_test(annual_Flow['max'])
print(mk_test)
print("\ntype of trend:", mk_test.trend, "(p-value: ", mk_test.p, ")")

In [ ]:
mk_test = mk.original_test(annual_Flow['q95'])
print(mk_test)
print("\n type of trend:", mk_test.trend, "(p-value: ", mk_test.p, ")")

# Flow duration curve

Finally let's calculate and plot the flow duration curve which is frequently used in hydrological analyses.  

Remember that the exceedence probability of given discharge can be estimated as:  

$$P[Q>Q_0] = \frac{m}{n+1}$$  

where $m$ is the rank of $Q_0$ when the $Q$ series is sorted from the largest to the smallest value and $n$ is the number $Q$ values

In [ ]:

Flow_dur = pd.DataFrame()
# sort the data:
Flow_dur['discharge'] = dailyStreamflow['00060_Mean'].sort_values(ascending = False)
# Add a column with the rank
Flow_dur['rank'] = range(1, len(Flow_dur)+1)
# calculate exceedence probability
Flow_dur['P_Exceed'] = 100* Flow_dur['rank']/(len(Flow_dur) + 1)

# And plot
plt.plot(Flow_dur['P_Exceed'], Flow_dur['discharge'],'k')
plt.yscale('log')
plt.xlabel('Exceedence probability')
plt.ylabel('Discharge')


# Export data

Each of the Data Frames generated can be exported to `csv` format using the method: `to_csv()`:

In [ ]:
Flow_dur.to_csv('Flow_duration_curve.csv')

# Import Data
A `csv` file can also be imported with the pandas function: `pd.read_csv()`:


In [ ]:
my_csv = pd.read_csv('Flow_duration_curve.csv')
my_csv

# Exercises
Prior to starting any of these notebooks, please make sure to save a copy of this to your google drive. This can be done by navigating to *file* > *Save a copy in Drive*. Your changes will not be saved unless this has been completed.

**NOTE: It is okay to work in groups to complete this assignment - but the assignment must be turned in and completed individually**

## Problem 1 - 2 points

1. Rerun this notebook to calculate flow trends at USGS gage 03293000. However, instead of plotting the q95, please plot the q50. Hint, under the section "Is there a trend?" Change `annual_Flow['q95'] = dailyStreamflow['00060_Mean'].resample('1Y').quantile(0.95)` to `annual_Flow['q50'] = dailyStreamflow['00060_Mean'].resample('1Y').quantile(0.50)`. Note, since you are changing `q95` to `q50`, all code below this referencing `q95` must also be changed to `q50`!
2. Please retitle the plot, "Discharge Trends" to "Discharge Trends 03293000 - MF Beargrass Creek at Old Cannons."
3. Please right click the plot, choose *save image as* and save the figure somewhere on your desktop. We will later bring the plot into Microsoft Word.

## Problem 2 - 3 points
Each of you have been assigned a USGS gage in Louisville, as shown in the table below.

Gage Number | Description | Student   
---------|-------|-------
03302000 | Pond Creek Near Louisville | Ellie, Lauren, Nick, Valerie  
03298000 | FLoyds Fork at Fisherville | Karina, McKenna, Noah

1. Under the section, "Get Data," please include a line of code that changes gage 03293000 to your assigned gage.
2. Please rerun the notebook to see flow trends at your assigned site (including qmin, qmax, qmean, and q50).
3. Please retitle the plot, "Discharge Trends" to "Discharge Trends [insert the number and name of your gage]."
4. Please right click the plot, choose *save image as* and save the figure somewhere on your desktop. We will later bring the plot into Microsoft Word.



## Problem 3 - 10 points
Please answer the following questions related to flow trends in Jefferson County in a separate Microsoft Word document.
1. Please show the figures plotting flow trends for USGS gage 03293000 (Middle Fork of Beargrass Creek at Old Cannons) and your assigned USGS gage.
2. How do flow trends differ between USGS gage 03293000 (Middle Fork of Beargrass Creek at Old Cannons) and your assigned USGS gage? In a concise paragraph, please hypothesize some potential reasons for differences in the trends. Note it may be helpful to gather information (e.g., watershed area, land use/land cover) on the watershed upstream of your gage from StreamStats.
3. What are potential mechanisms driving the trends that you may have observed both upstream of gage 03293000 and upstream of your assigned gage? Please explain your reasoning.
4. Noting that only three gages in Louisville have data dating back to the 1940s, to what degree could increased data collection improve understanding of long-term hydrologic trends?  
5. Please write a brief (i.e., ~ 3 sentence) reflection on how tools like Google Collab and Python can be used to conduct research/engineering work.  

## Problem 4
Please upload your Microsoft Word document as a PDF to Blackboard under the submission for Homework 3 part 3.